# Online Greedy Mixture DTW

This notebook implements an online greedy mixture-based DTW approach.  The only requirement in this notebook is that it implement the `offline_processing()` and `online_processing()` functions, which will be imported and run in `02_RunExperiment.ipynb`.  The rest of the notebook is for experimenting, visualizing, and analyzing the system, so it should be thought of as a sandbox for development.

Here is a summary of the online greedy mixture-based approach:
- We assume that the piano part starts in the correct location of the orchestra recording
- Chroma features are extracted from orchestra, piano, and full mix recordings
- Without constructing a 3D cost tensor, we compute a mixture-based cost (dissimilarity between P+O and PO) on-the-fly and greedily select the next step

Main findings:
- We did not find any promising results from this approach.

## Offline Processing

In the offline processing stage, three things are computed and stored in the `cache/` folder:
- chroma features for the orchestra recording
- chroma features for the full mix recording
- predicted DTW alignment between the orchestra and full mix recordings (not used)

In [ ]:
import numpy as np
import librosa as lb
import os
import os.path
import system_utils
import import_ipynb
import align_tools

In [ ]:
def offline_processing(scenario_dir, cache_dir, hop_length, steps, weights):
    '''
    Carries out offline processing for a simple offline DTW system.
    
    Inputs
    scenario_dir: The scenario directory to process
    cache_dir: The location of the cache directory
    hop_length: The hop length in samples used when computing chroma features
    steps: an L x 2 array specifying the allowable DTW transitions
    weights: a length L array specifying the DTW transition weights
    
    This function will store the computed chroma features and estimated alignment in the cache folder.
    '''
    
    # setup
    system_utils.verify_scenario_dir(scenario_dir)
    if os.path.exists(cache_dir):
        # print(f'{cache_dir} has already been processed.  Skipping.')
        pass
    else:
        # setup
        os.makedirs(cache_dir)

        # compute orchestra features
        o_file = f'{scenario_dir}/o.wav'
        y_o, sr = lb.core.load(o_file)
        F_o = lb.feature.chroma_cqt(y=y_o, sr=sr, hop_length=hop_length, norm=2) 

        # compute full mix features
        po_file = f'{scenario_dir}/po.wav'
        y_po, sr = lb.core.load(po_file)
        F_po = lb.feature.chroma_cqt(y=y_po, sr=sr, hop_length=hop_length, norm=2)
      
        # compute subsequence DTW alignment (orchestra as query) 
        orch_start_sec, orch_end_sec = system_utils.get_orchestra_start_end_times(scenario_dir)
        orch_start_frm = int(np.round(orch_start_sec * sr / hop_length))
        orch_end_frm = int(np.round(orch_end_sec * sr / hop_length)) + 1
        wp = align_tools.compute_dtw_alignment(1 - F_o[:,orch_start_frm:orch_end_frm].T @ F_po, steps, weights, subseq = True)
        wp[0,:] = wp[0,:] + orch_start_frm  # account for offset

        # save to cache
        np.save(f'{cache_dir}/o_chroma.npy', F_o)
        np.save(f'{cache_dir}/po_chroma.npy', F_po)
        np.save(f'{cache_dir}/o_po_align.npy', wp)
    
    return

In [ ]:
def verify_cache_dir(indir):
    '''
    Verifies that the specified cache directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/o_chroma.npy'), f'o_chroma.npy missing from {indir}'
    assert os.path.exists(f'{indir}/po_chroma.npy'), f'po_chroma.npy missing from {indir}'
    assert os.path.exists(f'{indir}/o_po_align.npy'), f'o_po_align.npy missing from {indir}'

# Online Processing

In the online processing stage, we do the following:
- We define a set of allowable transitions in a 3D cost tensor, with axes corresponding to P, O, and PO frames
- We do not actually allocate a 3D cost tensor.  Instead, we compute elements of this 3D tensor on-the-fly using a mixture-based cost and greedily select the next step to minimize cumulative cost.  The mixture-based cost indicates the dissimilarity between (a) the sum of piano and orchestra feature vectors, and (b) the full mix feature vector.



In [ ]:
def normalize_features(part_cqt):
    part_cqt_with_noise = part_cqt + np.abs(np.random.randn(*part_cqt.shape)) * 1e-8
    part_cqt_norm = part_cqt_with_noise / np.linalg.norm(part_cqt_with_noise, axis=0)

    mask = np.where(part_cqt_norm < np.max(part_cqt_norm) * 0.05)
    part_cqt_norm[mask] = 0

    return part_cqt_norm

In [ ]:
def online_processing(scenario_dir, out_dir, cache_dir, hop_sec, steps, weights):
    '''
    Carries out `online' processing for a simple offline DTW system.
    
    Inputs
    scenario_dir: The scenario directory to process
    out_dir: The directory to put results, intermediate files, and logging info
    cache_dir: The cache directory
    hop_length: The hop length in samples used when computing chroma features
    steps: an L x 2 array specifying the allowable DTW transitions
    weights: a length L array specifying the DTW transition weights

    This function will compute and save the predicted alignment in the output directory in a file hyp.npy
    '''
    
    # verify & setup
    system_utils.verify_scenario_dir(scenario_dir)
    verify_cache_dir(cache_dir)
    assert not os.path.exists(out_dir), f'Output directory {out_dir} already exists.'
    os.makedirs(out_dir)
    
    # compute features
    p_file = f'{scenario_dir}/p.wav'
    y, sr = lb.core.load(p_file)
    F_p = lb.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length, norm=2)  # piano features
    F_po = np.load(f'{cache_dir}/po_chroma.npy') # full mix features
    F_o = np.load(f'{cache_dir}/o_chroma.npy') # orchestra features
        
    # determine the start time of the query in the orchestra recording (ground truth)
    orch_start_sec, _ = system_utils.get_orchestra_query_boundaries(scenario_dir)
    orch_start_frm = orch_start_sec / hop_sec  # keep max precision, don't round
    
    # infer the start time of the query in the full mix recording (estimated)
    wp_BC_frm = np.flipud(np.load(f'{cache_dir}/o_po_align.npy'))
    wp_BC_frm = np.hstack((np.array([0,0]).reshape((2,-1)), wp_BC_frm)) # prepend (0,0) to handle edge cases properly
    wp_BC_sec = wp_BC_frm * hop_sec
    fullmix_start_sec = np.interp(orch_start_sec, wp_BC_sec[1,:], wp_BC_sec[0,:])

    F_o = F_o[:,int(orch_start_frm):]  # truncate orchestra features to start at query start
    F_po = F_po[:,int(fullmix_start_sec/hop_sec):]  # truncate full mix features to start at query start

    p_index = 0
    po_index = 0
    o_index = 0

    path = []

    steps = [
        [1, 1, 1],
        [1, 1, 2],
        [1, 2, 1],
        [2, 1, 1],
        [1, 2, 2],
        [2, 1, 2],
        [2, 2, 1],
    ]

    weights = [
        1,
        2,
        2,
        2,
        4,
        4,
        4,
    ]

    while (p_index < F_p.shape[1]) and (po_index < F_po.shape[1]) and (o_index < F_o.shape[1]):
        min_cost = np.inf
        min_step = None

        for step in steps:
            p_index_new = p_index + step[0]
            po_index_new = po_index + step[1]
            o_index_new = o_index + step[2]

            if (p_index_new >= F_p.shape[1]) or (po_index_new >= F_po.shape[1]) or (o_index_new >= F_o.shape[1]):
                continue

            cost = np.sum(weights[steps.index(step)] * (F_po[:, po_index_new] - (F_p[:, p_index_new] + F_o[:, o_index_new])))

            if cost < min_cost:
                min_cost = cost
                min_step = step

        if min_step is None:
            break
        
        p_index += min_step[0]
        po_index += min_step[1]
        o_index += min_step[2]

        path.append([p_index, po_index, o_index])

    path = np.array(path)

    wp_AC_sec = np.vstack((path[:,0], path[:,2])) * hop_sec
    np.save(f'{out_dir}/hyp.npy', wp_AC_sec)
    
    return

In [ ]:
def verify_hyp_dir(indir):
    '''
    Verifies that the specified scenario hypothesis directory has the required files.
    
    Inputs
    indir: The cache directory to verify
    '''
    assert os.path.exists(f'{indir}/hyp.npy')

# Example

Here is an example of how to call the offline and online processing functions on a scenario directory.

In [ ]:
# scenario_dir = 'scenarios/s2'
# out_dir = 'experiments/test/s2'
# cache_dir = 'experiments/test/cache'
# hop_size = 512
# steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
# weights = np.array([2,3,3])
# offline_processing(scenario_dir, cache_dir, hop_size, steps, weights)
# online_processing(scenario_dir, out_dir, cache_dir, hop_size, steps, weights)

## Debugging

Some code for debugging the system.  Can delete later.

In [ ]:
scenario_dir = 'scenarios/s2'
out_dir = 'experiments/debug/s2'
cache_dir = 'experiments/debug/cache'
hop_length = 512
steps = np.array([1,1,1,2,2,1]).reshape((-1,2))
weights = np.array([2,3,3])
hop_sec = 512 / 22050

In [ ]:
offline_processing(scenario_dir, cache_dir, hop_length, steps, weights)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# compute features
p_file = f'{scenario_dir}/p.wav'
y, sr = lb.core.load(p_file)
F_p = normalize_features(lb.feature.chroma_cqt(y=y, sr=sr, hop_length=hop_length, norm=2))  # piano features
F_po = normalize_features(np.load(f'{cache_dir}/po_chroma.npy')) # full mix features
F_o = normalize_features(np.load(f'{cache_dir}/o_chroma.npy')) # orchestra features

In [ ]:
# determine the start time of the query in the orchestra recording (ground truth)
orch_start_sec, _ = system_utils.get_orchestra_query_boundaries(scenario_dir)
orch_start_frm = orch_start_sec / hop_sec  # keep max precision, don't round

# infer the start time of the query in the full mix recording (estimated)
wp_BC_frm = np.flipud(np.load(f'{cache_dir}/o_po_align.npy'))
wp_BC_frm = np.hstack((np.array([0,0]).reshape((2,-1)), wp_BC_frm)) # prepend (0,0) to handle edge cases properly
wp_BC_sec = wp_BC_frm * hop_sec
fullmix_start_sec = np.interp(orch_start_sec, wp_BC_sec[1,:], wp_BC_sec[0,:])

F_o = F_o[:,int(orch_start_frm):]  # truncate orchestra features to start at query start
F_po = F_po[:,int(fullmix_start_sec/hop_sec):]  # truncate full mix features to start at query start

In [ ]:
custom_cost = np.zeros((F_p.shape[1], F_o.shape[1]))

for i in range(F_p.shape[1]):
    for j in range(F_o.shape[1]):
        custom_cost[i,j] = np.dot(F_po[:, j], (F_p[:, i] + F_o[:, j])) / (np.linalg.norm(F_po[:, i]) + np.linalg.norm(F_p[:, i] + F_o[:, j]))

custom_cost = 1-custom_cost/np.max(custom_cost)

plt.imshow(custom_cost, origin = 'lower')
plt.colorbar()
plt.show()

In [ ]:
p_index = 0
o_index = 0

path = []

steps = [
    [1, 1, 2],
    [1, 2, 1],
    [2, 1, 1],
]

while (p_index < F_p.shape[1]) and (o_index < F_o.shape[1]):
    min_cost = np.inf
    min_step = None

    for step in steps:
        p_index_new = p_index + step[0]
        o_index_new = o_index + step[1]

        if (p_index_new >= F_p.shape[1]) or (o_index_new >= F_o.shape[1]):
            continue

        cost = custom_cost[p_index_new, o_index_new] * step[2]

        if cost < min_cost:
            min_cost = cost
            min_step = step

    if min_step is None:
        break

    p_index += min_step[0]
    o_index += min_step[1]

    path.append([p_index, o_index])

path = np.array(path)

In [ ]:
plt.plot(path[:,1], path[:,0], color='r')
plt.imshow(custom_cost, origin = 'lower')
plt.show()

In [ ]:
wp_AC_sec = path.T * hop_sec
wp_AC_sec[1,:] += orch_start_sec
np.save(f'{out_dir}/hyp.npy', wp_AC_sec)

In [ ]:
p_index = 0
po_index = 0
o_index = 0

path = []
step_path = []

steps = [
    [1, 1, 1, 1],
    [1, 1, 2, 1],
    [1, 2, 1, 1],
    [2, 1, 1, 1],
    [1, 2, 2, 1],
    [2, 1, 2, 1],
    [2, 2, 1, 1],
]

while (p_index < F_p.shape[1]) and (po_index < F_po.shape[1]) and (o_index < F_o.shape[1]):
    min_cost = np.inf
    min_step = None

    rand_steps = np.random.permutation(steps).tolist()

    for step in rand_steps:
        p_index_new = p_index + step[0]
        po_index_new = po_index + step[1]
        o_index_new = o_index + step[2]

        if (p_index_new >= F_p.shape[1]) or (po_index_new >= F_po.shape[1]) or (o_index_new >= F_o.shape[1]):
            continue

        cost = step[3] * np.linalg.norm(F_po[:, po_index_new] - (F_p[:, p_index_new] + F_o[:, o_index_new]))

        if cost < min_cost:
            min_cost = cost
            min_step = step

    if min_step is None:
        break
    
    p_index += min_step[0]
    po_index += min_step[1]
    o_index += min_step[2]

    path.append([p_index, po_index, o_index])
    step_path.append(steps.index(min_step))

path = np.array(path)

wp_AC_sec = np.vstack((path[:,0], path[:,2])) * hop_sec
wp_AC_sec[1,:] += orch_start_sec
np.save(f'{out_dir}/hyp.npy', wp_AC_sec)

In [ ]:
plt.plot(wp_AC_sec[0,:], wp_AC_sec[1,:])
plt.grid()

In [ ]:
plt.hist(step_path, bins=range(len(steps)+1))
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=3, sharex=True, sharey=True, figsize=(15, 7))
lb.display.specshow(F_o, y_axis='chroma', x_axis='time', ax=ax[0])
ax[0].set(title='o')
ax[0].label_outer()
img = lb.display.specshow(F_p, y_axis='chroma', x_axis='time', ax=ax[1])
ax[1].set(title='p')
ax[1].label_outer()
img = lb.display.specshow(F_po, y_axis='chroma', x_axis='time', ax=ax[2])
ax[2].set(title='po')
fig.colorbar(img, ax=ax)
plt.show()